In [41]:
!pip install deep-translator spacy unidecode nltk
!python -m spacy download en_core_web_sm

import re

import nltk
import spacy
from deep_translator import GoogleTranslator
from nltk.corpus import stopwords
from unidecode import unidecode

import services.influencers_sevice as influencers_service

^C


In [42]:
# Load necessary libraries
nltk.download('stopwords')
stopwords_en = stopwords.words('english')
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
translator = GoogleTranslator(source='auto', target='en')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wawao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
def clean_and_process_posts(posts):
    """ This performs URL removal, special character cleaning, emoji removal, translation, stopword removal, and lemmatization."""

    for post in posts:
        print(post)
        content = post.get('content', '')
        try:
            # Removing URLs
            content = re.sub(r'http\S+', '', content)

            # Removing Special Characters
            content = re.sub(r'\W+', ' ', content)

            # Removing Emojis
            content = unidecode(content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this caption.")

        try:
            # Translating Text
            content = translator.translate(content)
        except Exception as e:
            print(f"Translation error: {e}. Skipping translation.")

        # Removing Stopwords
        words = content.split()
        content = ' '.join([word for word in words if word not in stopwords_en])

        # Lemmatizing Text
        doc = nlp(content)
        lemmatized_content = " ".join([token.lemma_ for token in doc])
        print(lemmatized_content)
        # Update the post with cleaned content
        try:
            influencers_service.update_post(post, 'content', lemmatized_content)
        except Exception as e:
            print(f"Error updating post: {e}.")


In [44]:
# Load posts
posts = influencers_service.get_posts()

# Clean and process posts
clean_and_process_posts(posts)